In [10]:
import json
import glob
import itertools

import numpy as np
import pandas as pd

from qiskit import transpile, QuantumCircuit
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp


import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import (
    GCNConv, 
    TransformerConv, 
    GATv2Conv, 
    global_mean_pool, 
    Linear, 
    ChebConv, 
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem
from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer.primitives import Estimator as AerEstimator

from pprint import pprint

In [24]:
AerEstimator().run()

TypeError: BaseEstimator.run() missing 2 required positional arguments: 'circuits' and 'observables'

In [11]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)


In [12]:
properties

{'name': 'fake_lima',
 'gates_set': ['x', 'reset', 'sx', 'id', 'cx', 'rz'],
 'num_qubits': 5,
 'qubits_props': {0: {'index': 0,
   't1': 5.9698643286635694e-05,
   't2': 9.355584184359312e-05,
   'readout_error': 0.026100000000000012},
  1: {'index': 1,
   't1': 8.305997230317399e-05,
   't2': 0.00011553074510239035,
   'readout_error': 0.020000000000000018},
  2: {'index': 2,
   't1': 0.00010377694598809795,
   't2': 9.477169960638749e-05,
   'readout_error': 0.016599999999999948},
  3: {'index': 3,
   't1': 4.358447375590962e-05,
   't2': 4.645933441447346e-05,
   'readout_error': 0.0515000000000001},
  4: {'index': 4,
   't1': 1.7543975812787366e-05,
   't2': 1.6441110002077736e-05,
   'readout_error': 0.057499999999999996}},
 'gate_props': {'id_0': {'index': 'id_0',
   'gate_error': 0.00019195510390342677,
   'gate_length': 35.55555555555556},
  'id_1': {'index': 'id_1',
   'gate_error': 0.00033064681663890665,
   'gate_length': 35.55555555555556},
  'id_2': {'index': 'id_2',
   'g

## dataset generation: post transpilation depth

In [13]:
def save_data(depth: int, data: dict):
    for bucket, entries in data.items():
        with open(f"./data/uniform_distr_dataset/post_transpilation_depth/5_qubits/random_fixed/qasm_{depth}_{bucket}.txt", "w") as f:
            lines = []
            for (qasm, exp_val) in entries:
                lines.append("exp_val:\n")
                lines.append(str(exp_val)+"\n")
                lines.append("qasm:\n")
                lines.append(qasm)
                lines.append("-----\n")

            f.writelines(lines)
    return f"./data/uniform_distr_dataset/post_transpilation_depth/5_qubits/random_fixed/qasm_{depth}..."


In [ ]:
generated_data = {}
for post_transpile_depth in range(1, 20):
    generated_data[post_transpile_depth] = {
        round(x * 0.1, 1): []
        for x in range(-10, 10, 1)
    }

obs = SparsePauliOp(["ZZZZZ"])
encoded_obs = encode_pauli_sum_op(obs)

CIRCUITS_PER_BIN = 100
counter = 0
condition = len(generated_data.keys()) > 0

aer_estimator = AerEstimator()

while condition:
    counter += 1
        
    circuit = transpile(
        random_circuit(5, 1, measure=True),
        backend,
        optimization_level=0,
    )

    depth = circuit.depth(lambda x: x[0].num_qubits == 2)
    
    # if depth already filled
    if depth not in generated_data:
        continue
    
    ideal_exp_vals = create_meas_data_from_estimators(
        circuits=circuit, observables=obs, estimators=[aer_estimator]
    )
    ideal_exp_val = ideal_exp_vals[0]
    round_exp_val = round(ideal_exp_val, 1)
        
    qasm_str = circuit.qasm()
    
    target_bin = round(ideal_exp_val, 1)
    
    if round_exp_val not in generated_data[depth]:
        continue
        
    if len(generated_data[depth][round_exp_val]) >= CIRCUITS_PER_BIN:
        continue
    
    # add data
    generated_data[depth][target_bin].append((qasm_str, ideal_exp_val))
            
    if all([
        len(b) >= CIRCUITS_PER_BIN - 10 for b in generated_data[depth].values()
    ]):
        path = save_data(depth, generated_data[depth])
        print(f"Saved to {path}")
        
        del generated_data[depth]

    if counter % 500 == 0 and counter > 0:
        display = {d: [len(db) for db in b.values()] for d, b in generated_data.items()}
        
        print(counter)
        pprint(display)

500
{1: [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 2: [10, 0, 1, 0, 1, 3, 0, 1, 4, 3, 28, 4, 1, 1, 0, 1, 4, 4, 0, 1],
 3: [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 4: [1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 11, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 5: [16, 2, 1, 3, 1, 0, 1, 1, 3, 6, 43, 5, 2, 1, 2, 2, 3, 1, 1, 4],
 6: [0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 10, 1, 0, 0, 0, 0, 0, 1, 1, 1],
 7: [2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 14, 0, 1, 1, 2, 0, 0, 0, 0, 0],
 8: [5, 0, 1, 1, 0, 0, 0, 0, 1, 2, 11, 0, 0, 1, 1, 1, 1, 0, 1, 2],
 9: [1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 6, 1, 0, 0, 0, 0, 1, 1, 0, 0],
 10: [2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 11: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 12: [1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 13: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 14: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 15: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0

### convert to consumable format

In [17]:
observable = encode_pauli_sum_op(SparsePauliOp("ZZZZZ"))

for depth in tqdm_notebook(list(range(1, 19))):
    
    for zne_path in glob.glob(
        f"./data/uniform_distr_dataset/post_transpilation_depth/5_qubits/results/lima_zne/qasm_{depth}_**.txt"
    ):
        filename = zne_path.split("/")[-1]
        
        fine_postfix = filename.split("qasm_")[-1].split(".txt")[0]
        entries = []

        depth = filename.split("_")[1]
        
        zne_results = []
        with open(zne_path, "r") as zne_file:
            zne_results = [float(r) for r in zne_file.read().split("\n")[:-1]]

        default_results = []
        with open(
            f"./data/uniform_distr_dataset/post_transpilation_depth/5_qubits/results/lima_default/{filename}", 
            "r"
        ) as default_file:
            default_results = [float(r) for r in default_file.read().split("\n")[:-1]]
            
        with open(f"./data/uniform_distr_dataset/post_transpilation_depth/5_qubits/random_fixed/{filename}", "r") as qasm_file:
            file_data_entry = qasm_file.read().split("-----")[:-1]

            ideal_results = []

            for idx, file_data in enumerate(file_data_entry):
                head, tail = file_data.split("qasm:\n")

                qasm_str = tail.strip()
                ideal_exp_val = float(head.split("exp_val:\n")[-1].strip())

                ideal_results.append(ideal_exp_val)
                
                circuit = QuantumCircuit.from_qasm_str(qasm_str)
                
                graph_data = circuit_to_graph_data_json(
                    circuit=circuit,
                    properties=properties,
                    use_qubit_features=True,
                    use_gate_features=True,
                )
                
                try:
                    entry = ExpValueEntry(
                        circuit_graph=graph_data,
                        observable=observable,
                        ideal_exp_value=ideal_exp_val,
                        noisy_exp_values=[
                            default_results[idx],
                            zne_results[idx]
                        ],
                        circuit_depth=circuit.depth()
                    )

                    entries.append(entry.to_dict())
                except:
                    print(f"passing {idx}")
                    pass
    
        save_file_path = f"./data/uniform_distr_dataset/post_transpilation_depth/5_qubits/pyg_data/{fine_postfix}.json"
        with open(save_file_path, "w") as json_file:
            print(f"Saving {save_file_path}...")
            json.dump(entries, json_file)

  0%|          | 0/18 [00:00<?, ?it/s]

KeyboardInterrupt: 